In [82]:
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from scipy.stats import zscore
from scipy import stats
import plotly.express as px
import numpy as np
import operator
from sklearn.preprocessing import normalize
import scipy.cluster.hierarchy as shc
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
import umap.umap_ as umap
import random
import itertools
import os 

In [83]:
def dataframe_znorm_differenz(dataframe):
    
    # identifier 'T' for Vitalwert and Labor --> change 'T' to 'Temperatur' for Labor
    # then delete these values (don't make sense!)
    dataframe.loc[(dataframe.Wertbezeichner=='T')&(dataframe.Kategorie=='Labor'), 'Wertbezeichner'] = 'Temperatur'
    dataframe = dataframe[dataframe.Wertbezeichner!='Temperatur']
    # apply z-normalization
    dataframe['Wert_norm'] = None
    dataframe.loc[dataframe.Kategorie=='Vitalwert', 'Wert_norm'] = dataframe[dataframe['Kategorie']=='Vitalwert'].groupby(['Wertbezeichner'])['Wert'].transform(lambda x : zscore(x,ddof=0))
    dataframe.loc[dataframe.Kategorie=='Labor', 'Wert_norm'] = dataframe[dataframe['Kategorie']=='Labor'].groupby(['Wertbezeichner'])['Laborwert'].transform(lambda x : zscore(x,ddof=0)) 
    temp = dataframe[dataframe['Wertbezeichner']=='RR']['Mean']
    dataframe.loc[dataframe.Wertbezeichner=='RR', 'Wert_norm'] = (temp - temp.mean())/temp.std(ddof=0)
    
    # get difference of 'Einfuhr' and 'Ausfuhr'
    df_ein_temp = dataframe[dataframe['Wertbezeichner']=='Einfuhr']
    df_ein = df_ein_temp['Wert'].astype(int)
    df_aus_temp = dataframe[dataframe['Wertbezeichner']=='Ausfuhr']
    df_aus = df_aus_temp['Wert'].astype(int)
    
    df_diff = df_ein.values - df_aus.values
    dataframe['Differenz'] = None
    dataframe.loc[dataframe.Wertbezeichner=='Einfuhr', 'Differenz'] = df_diff
    dataframe.loc[dataframe.Wertbezeichner=='Ausfuhr', 'Differenz'] = -df_diff
    dataframe['Differenz'] = dataframe['Differenz'].astype(float)
    dataframe['Wert_norm'] = dataframe['Wert_norm'].astype(float)

    return dataframe

In [84]:
# there might be missing values (e.g. 2018)
def check_bilanz(dataframe_list):
    for df_index in range(len(dataframe_list)):
        not_one_list = True
        while not_one_list:
            dataframe = dataframe_list[df_index]
            df_ein_temp = dataframe[dataframe['Wertbezeichner']=='Einfuhr']
            df_ein = df_ein_temp['Wert'].astype(int)
            df_aus_temp = dataframe[dataframe['Wertbezeichner']=='Ausfuhr']
            df_aus = df_aus_temp['Wert']
            diff_index = [x1 - x2 for (x1, x2) in zip(list(df_ein.index), list(df_aus.index))]
            not_one = [i for i in diff_index if i != 1]
            if not_one != []:
                first_difference = diff_index.index(not_one[0])
                i = list(df_ein.index)[first_difference]
                dataframe = dataframe.drop(i)
                dataframe_list[df_index] = dataframe
            else:
                not_one_list = False
    return dataframe_list

In [96]:
def get_cluster_matrix(df, cat='Vitalwert', days_interval=5, test_pat=34035669, hour=False, nan_to_zero=True):
    x = None
    # list of all the patient numbers in the given file df
    patient_list = sorted(df.FallNr.unique())
    # list of all the identifiers of category 'cat' in the given file df (sorted alphabetically)
    ident_list = sorted(df[df.Kategorie==cat].Wertbezeichner.unique()) 
    if hour == True:
        X = np.zeros((len(patient_list),len(ident_list)*days_interval*24))
        hour_list = list(range(0,24))
    else:
        X = np.zeros((len(patient_list),len(ident_list)*days_interval))   
    for pat in patient_list:
        if list(patient_list).index(pat) % 50 == 0:
            print(list(patient_list).index(pat), ' of ', len(patient_list))
        df_pat = df[(df.FallNr==pat)&(df.Kategorie==cat)]
        # get dataframe with date and identifier as multiindex and mean over values of corresponding column values
        if cat=='Vitalwert' and hour==True:
            # array where the data will be saved in (what we actually want)
            df_pat_temp = df_pat.groupby([df_pat['Wertbezeichner'], df_pat['Zeitstempel'].dt.date, df_pat['Zeitstempel'].dt.hour]).mean()
            
            if pat == test_pat:
                x = df_pat_temp
            # date of first measurement
            first_date = list(df_pat['Zeitstempel'].dt.date)[0] # frühestes Datum
            # define 'maximal date' of interval of days we want to look at
            last_date = first_date + dt.timedelta(days=days_interval-1)
            # get list with all dates between first_date and last_date (even if not in df!)
            # date_list = [first_date, first_date+1, first_date+2, first_date+3, last_date]
            # even if e.g. at first_date+1 nothing was measured
            date_list = [first_date + dt.timedelta(days=x) for x in range(days_interval)]
            pat_ident = df_pat_temp.index.get_level_values(0).unique()
           # all identifiers
            if sorted(pat_ident) != sorted(ident_list):
                diff = list(set(pat_ident).symmetric_difference(set(ident_list)))
                for elem in diff:
                    temp = list(itertools.chain.from_iterable(itertools.repeat(x, 24) for x in date_list))

                    index = pd.MultiIndex.from_tuples(list(zip([elem]*days_interval*24, temp, hour_list*len(date_list))), names=["Wertbezeichner", "Zeitstempel", "Stunden"])
                    s = pd.DataFrame(np.zeros((days_interval*24, df_pat_temp.shape[1])), index=index) 
                    s[0] = np.array([pat]*days_interval*24)
                    for da in date_list:
                        for h in hour_list:
                            df_pat_temp.loc[(elem, da, h), :] = s
                df_pat_temp = df_pat_temp.sort_index()
            value_list = []
            for i in ident_list:
                df_temp = df_pat_temp.loc[i].loc[first_date:last_date]
                index_list_dates = []

                for j in range(len(df_temp.index)):
                    index_list_dates.append(df_temp.index[j][0])
                index_list_dates = list(set(index_list_dates))

                # all dates
                if list(sorted(index_list_dates)) != sorted(date_list):
                    temp = np.zeros(df_temp.shape[1])
                    temp[0] = np.array([pat])
                    diff = list(set(list(index_list_dates)).symmetric_difference(set(date_list)))
                    for elem in diff:
                        for h in hour_list:
                            df_pat_temp.loc[(i, elem, h), :] = temp
                    df_pat_temp = df_pat_temp.sort_index()
                # all hours
                for day in sorted(date_list):
                    index_list_hours = list(df_pat_temp.loc[i].loc[day].index)
                    if list(sorted(index_list_hours)) != sorted(hour_list):
                        temp = np.zeros(df_temp.shape[1])
                        temp[0] = np.array([pat])
                        diff = list(set(list(index_list_hours)).symmetric_difference(set(hour_list)))
                        for h in diff:
                            df_pat_temp.loc[(i, day, h), :] = temp
                        df_pat_temp = df_pat_temp.sort_index()
                df_temp = df_pat_temp.loc[i][first_date:last_date]['Wert_norm']
                value_list.extend(df_temp)
        else:
            # array where the data will be saved in (what we actually want)
            df_pat_temp = df_pat.groupby([df_pat['Wertbezeichner'], df_pat['Zeitstempel'].dt.date]).mean()
            if pat == test_pat:
                x = df_pat_temp
            # date of first measurement
            first_date = list(df_pat['Zeitstempel'].dt.date)[0] # frühestes Datum
            # define 'maximal date' of interval of days we want to look at
            last_date = first_date + dt.timedelta(days=days_interval-1)
            # get list with all dates between first_date and last_date (even if not in df!)
            # date_list = [first_date, first_date+1, first_date+2, first_date+3, last_date]
            # even if e.g. at first_date+1 nothing was measured
            date_list = [first_date + dt.timedelta(days=x) for x in range(days_interval)]
            pat_ident = df_pat_temp.index.get_level_values(0).unique()

            # all identifiers
            if sorted(pat_ident) != sorted(ident_list):
                print('here')
                diff = list(set(pat_ident).symmetric_difference(set(ident_list)))
                for elem in diff:
                    index = pd.MultiIndex.from_tuples(list(zip([elem]*days_interval, date_list)), names=["Wertbezeichner", "Zeitstempel"])
                    s = pd.DataFrame(np.zeros((days_interval, df_pat_temp.shape[1])), index=index) 
                    s[0] = np.array([pat]*days_interval)
                    for da in date_list:
                        df_pat_temp.loc[(elem, da), :] = s
                df_pat_temp = df_pat_temp.sort_index()
            value_list = []
            for i in ident_list:
                print(i)
                df_temp = df_pat_temp.loc[i].loc[first_date:last_date]
                # all dates
                if list(sorted(df_temp.index)) != sorted(date_list):
                    temp = np.zeros(df_temp.shape[1])
                    temp[0] = np.array([pat])
                    diff = list(set(list(df_temp.index)).symmetric_difference(set(date_list)))
                    for elem in diff:
                        df_pat_temp.loc[(i, elem), :] = temp
                    df_pat_temp = df_pat_temp.sort_index()
                df_temp = df_pat_temp.loc[i][first_date:last_date]['Wert_norm']
                value_list.extend(df_temp)
            
        idx = list(patient_list).index(pat)
        X[idx] = np.array(value_list)
    if nan_to_zero == True:
        # replace all nans with zero!!!!!
        X[np.isnan(X)] = 0
    return X, x, ident_list

In [86]:
def remove_outliers(df):
    print(df.shape)
    # Temperatur: max 50°C, ging bis 3582°C
    df = df.drop(df[(df.Wertbezeichner == 'T') & (df.Wert > 50)].index)
    print(df.shape)
    # RR: Mean: min=0, max=755, danach alles < 300. Ich cute den extremen Ausreißer
    df = df.drop(df[(df.Mean > 750)].index)
    print(df.shape)
    
    # AF: zu wenig Ahnung. min=0, max=155, ber 30 gilt als erhöht, hier gibt es Werte 100+
    # HF: zu wenig Ahnung. min=0, max=270
    # HF_Pulsoxy: zu wenig Ahnung. min=22, max=300

    # SPO2: min = 5, max=100
    return df

In [87]:
def save_file(file, filename):
    name = filename + '_' + str(file.shape[0]) + '_'  + str(file.shape[1]) + '.txt'
    a_file = open(name, 'w')
    for row in file:
        np.savetxt(a_file, row)

    a_file.close()

In [88]:
path='./data/'
df_list = []
for file in os.listdir(path):
    if '_modified_80' in file and '#' not in file:
        print(file)
        with open(os.path.join(path, file), 'r+') as f:
            df_temp = pd.read_csv(f, error_bad_lines=False, warn_bad_lines=False, sep = ';', parse_dates=['Zeitstempel'])
            df_list.append(df_temp)
            
print(len(df_list))

Daten_SAB_COPRA_2016_modified_80.csv
Daten_SAB_COPRA_2015_modified_80.csv
Daten_SAB_COPRA_2018_modified_80.csv
Daten_SAB_COPRA_2017_modified_80.csv
Daten_SAB_COPRA_2014_modified_80.csv
Daten_SAB_COPRA_2019_modified_80.csv
6


In [89]:
df_list = check_bilanz(df_list)
df = pd.concat(df_list)  
df = df.reset_index(drop=True)
df = remove_outliers(df)
df = dataframe_znorm_differenz(df)

(1668731, 10)
(1668725, 10)
(1668724, 10)


In [90]:
df.to_csv(path+'Daten_COPRA_all_subset_80.csv', sep=';')

In [91]:
pat_list = df.FallNr.unique()
len(pat_list)

294

In [92]:
days_interval = 5
test_pat = random.choice(sorted(df.FallNr.unique()))

# Vitalwerte

all Patients, mean over day

In [93]:
X_vital_all, x_vital_all, vital_list_all = get_cluster_matrix(df, days_interval=days_interval, test_pat=test_pat, hour=False)
save_file(X_vital_all, 'X_vital_all')

0  of  294
50  of  294
100  of  294
150  of  294
200  of  294
250  of  294


In [94]:
print('first dimension should be:', len(pat_list), '(= number of feasible patients)')
print('second dimension should be:', days_interval * len(df[df.Kategorie=='Vitalwert']['Wertbezeichner'].unique()), '(= days_interval * number of identifiers for category Vitalwert)')
print('actual dimensuons are', X_vital_all.shape)

first dimension should be: 294 (= number of feasible patients)
second dimension should be: 35 (= days_interval * number of identifiers for category Vitalwert)
actual dimensuons are (294, 35)


all patients, mean over hours

In [99]:
X_vital_all_hour, x_vital_all_hour, vital_list_all_hour = get_cluster_matrix(df, days_interval=days_interval, test_pat=test_pat, hour=True)
save_file(X_vital_all_hour, 'X_vital_all_hour')

0  of  294
50  of  294
100  of  294
150  of  294
200  of  294
250  of  294


In [100]:
print('first dimension should be:', len(pat_list), '(= number of feasible patients)')
print('second dimension should be:', days_interval * len(df[df.Kategorie=='Vitalwert']['Wertbezeichner'].unique()) * 24, '(= days_interval * number of identifiers for category Vitalwert * hours in a day)')
print('actual dimensuons are', X_vital_all_hour.shape)

first dimension should be: 294 (= number of feasible patients)
second dimension should be: 840 (= days_interval * number of identifiers for category Vitalwert * hours in a day)
actual dimensuons are (294, 840)


# Laborwerte

all patients, mean over day

In [97]:
X_labor_all, x_labor_all, labor_list_all = get_cluster_matrix(df, cat='Labor', days_interval=days_interval, test_pat=test_pat, hour=False)
save_file(X_labor_all, 'X_labor_all')

0  of  294
here
% unreife Thr.
- ADP [5 µM]
- ADP-Revers.
- Arach. [0.5mM]
- Epineph.[5 µM]
- Koll.[10µg/ml]
- PFA Coll./ADP
- PFA Coll./Epi.
- a. Ly. v. reak
-Albumin
-Albumin abs.
-Albumin in Liq.
-Albumin in S.
-Alpha-1-Glob.
-Alpha-1-Glob.ab
-Alpha-2-Glob.
-Alpha-2-Glob.ab
-Anisozytose
-Atyp. Lympho.% man
-Atyp. Zellen % man
-Bakterien
-Basophile %
-Basophile % man
-Basophile abs.
-Beta-Glob.
-Beta-Glob. abs.
-Blasten % man
-Blut (Ery.)
-Dichte
-EVB
-Eiweiß
-Eosinophile %
-Eosinophile % man
-Eosinophile abs
-Epithelien
-Erythrobl. % man
-Erythrozyten
-Gamma-Glob. 3182
-Gamma-Glob. abs 3183
-Hefe mikr.
-Hyal. Zyl.
-Hypochromasie
-Hämatokrit
-Hämoglobin
-IGA in Liquor
-IGA in Serum
-IGG Index
-IGG in Liquor
-IGG in Serum
-IGM in Liquor
-IGM in Serum
-Kernsch.in100%
-Leukos
-Leukozyten
-Lymphozyten %
-Lymphozyten % man
-Lymphozyten abs
-MCH
-MCHC
-MCV
-MTV
-Makrozytose
-Metamyeloz. % man
-Mikrozytose
-Monozyten %
-Monozyten % man
-Monozyten abs.
-Myelozyten % man
-Neutrophile %
-Neutr

Thrombozyten Cit HA
Thrombozyten PRP HA
Transf.-Sättig.
Transf.reaktion TR
Transferrin
Triglyceride
Troponin-Ths
U-MikroALB
U-MikroALB 2
UFH-Hep. HA
Urin Sammelvol.
Urin Sammelzeit
VZV-GS VR
VZV-MS VR
Valproinsäure
Vancomycin
Vitamin B12
Zytoz.präp. Liq.
a/ApO2
aPTT (FS,CS5100)
aPTT FS HA
aPTT neu
aPTT570nm HA
alpha-HBDH 37C
dRVV Ratio norm.
extrazel.Hb HA
gamma-GT IFCC
korr. rel. Reti.
mOsm
p50(act)
pCO2
pCO2(T)
pH
pH(T)
pO2
pO2(T)
proBNP
sO2
sTrf-Rezeptor
tHb
tO2
vWF Akt.(CS5100)
vWF-Ag (CS5100)
vWF-Ag neu HA
vWF-Akt.n HA
vWF-Ratio
vWF-Ratio HA
here
% unreife Thr.
- ADP [5 µM]
- ADP-Revers.
- Arach. [0.5mM]
- Epineph.[5 µM]
- Koll.[10µg/ml]
- PFA Coll./ADP
- PFA Coll./Epi.
- a. Ly. v. reak
-Albumin
-Albumin abs.
-Albumin in Liq.
-Albumin in S.
-Alpha-1-Glob.
-Alpha-1-Glob.ab
-Alpha-2-Glob.
-Alpha-2-Glob.ab
-Anisozytose
-Atyp. Lympho.% man
-Atyp. Zellen % man
-Bakterien
-Basophile %
-Basophile % man
-Basophile abs.
-Beta-Glob.
-Beta-Glob. abs.
-Blasten % man
-Blut (Ery.)
-Dichte
-EVB


LDL-Chol.
LH 2
Lac
Lactat
Lactat i. Liq.
Lambda-Leichtk.
Leukoz. i. L.
Lipase 37C
Liquor:
Magnesium i. U.
Makr. Beurt. Liq
Mediz. Hinweise
MetHb
Metan. in EDTA
Mg
Mg i.U. Ausch.
MixCon C
NRBCs absolut
NSE
Na
Na i. U. Aussch.
Na+
Natrium i. U.
Normetanephrin
Nucleated RBC %
O2Hb
Osmo. i.S. gem.
Osmo. i.U. gem.
Osmolalit.i.U.-a
P, anorg.
PAI AG
PFA ADP HA
PFA EPI HA
PO4 i.U. Aussch.
PRI-Wert HA
Phenytoin CH
Phosphat i. U.
Plasmin-Inhib. HA
Plasminogen HA
Procalcitonin
Prolaktin basal
Prot. S frei Ag
Prot. S ges. Ag
Protein C
Protein C Ag
Protein C Akt.
Protein S
Protein S Akt.
Protein S-Akt.CC HA
Protein S-Akt.St
ProteinC-Akt.ch.
Quick
Quick (CS5100)
Quick HA
Quick570nm HA
Quot. Aldo-Renin
RHb
RI
Reifeindex Reti.
Renin 2
Retik.-Hb-Äquiv.
Retik.Prod.Index
Retikulozyt. abs
Retikulozyten %
Rhin-RNA VR
S-100
SBC
SBE
TAFI HA
TRAP Citrat HA
TSH
TSH-Rez. AK
TZ (CS5100)
TZ HA
Theophyllin
Thrombinzeit
Thromboz.Citr. CH
Thrombozyten Cit HA
Thrombozyten PRP HA
Transf.-Sättig.
Transf.reaktion TR
Tra

Fibrinogen Konz.
Freies T4
G.Eiw.i.U. Aus.
G.Eiw.i.U. zu Cr
GFR-CKD-EPI
GFR-IFCC
GFR-korr
GOT (AST) IFCC
GPT (ALT) IFCC
Gentamicin
Ges.-Eiw.i.U.
Gesamt-T3
Gesamt-T4
Gesamteiweiß
Glu
Glucose
Glucose i. Liq.
Glucose i. S./LH
Glucose im Urin
HCG u. ß-HCG
HCO3-
HDL-Chol.
HIT-II Hep.neut.
HIT-II OD
HST i. U. Aus.
Haptoglobin 2
Harnstoff
Harnstoff i.U.
Harnstoff-Clear.
Harnsäure
HbA1c (DCCT)
HbA1c (IFCC)
Hct
I-Thrombo-EDTA HA
IGA 2
IGF-1 3728
IGG 2
IGM 2
IL-6
INR
INR (CS5100)
INR HA
INR570nm HA
INTEM_Alpha Rotem
INTEM_Alpha10 Rotem
INTEM_CFT Rotem
INTEM_CT Rotem
IgG HA
IgM HA
K
K i. U. Aussch.
K+
Kalium i. U.
KaocLot HA
Kappa - Leichtk.
Kappa/Lambda-Qu.
Kollagen-10µg/ml HA
Körper-Oberfl.
Körpergewicht
Körpergröße
LDH IFCC
LDL-Chol.
LH 2
Lac
Lactat
Lactat i. Liq.
Lambda-Leichtk.
Leukoz. i. L.
Lipase 37C
Liquor:
Magnesium i. U.
Makr. Beurt. Liq
Mediz. Hinweise
MetHb
Metan. in EDTA
Mg
Mg i.U. Ausch.
MixCon C
NRBCs absolut
NSE
Na
Na i. U. Aussch.
Na+
Natrium i. U.
Normetanephrin
Nucleated RBC %


CK-MB 37C
CK-MB rel.
CMV-GS VR
COHb
CRP
Ca
Ca i. U. Aussch.
Ca++
Ca, korr. (Alb.)
Calcium i. U.
Chlorid i. U.
Chlorid i.U. Aus
Cholesterin
Cl
Cl-
Cortisol i. S.
Creatinin
Creatinin i.U.
Creatinin-Clear.
Cut Off HA
Cystatin C IFCC neu
D-Dimer
D-Dimer (Innov.)
D-Dimer (high)
D-Dimere (Innov) HA
D-Dimere vd 1:20 HA
Deseth.amiodaron
Digitoxin
Digoxin
EBV-DNA VR
ERY i. L.
EXTEM_Alpha Rotem
EXTEM_Alpha10 Rotem
EXTEM_CFT Rotem
EXTEM_CT Rotem
Eiw. Liquor
Epinephrin (5µM) HA
Ethanol
Extinktion HA
F V Leiden
F. VIII:C - high
F. XIII (CS5100)
FIBTEM_Ampl10 HA
FIO2
FSH 2
FT3
FVIII high HA
Faktor II HA
Faktor IX HA
Faktor V HA
Faktor VII HA
Faktor X HA
Faktor XI HA
Faktor XII HA
Faktor XIII HA
Fe
Ferritin 2
Ferritin-Index
Fib.n.C.(CS5100)
Fib/C HA
Fibrinogen
Fibrinogen Konz.
Freies T4
G.Eiw.i.U. Aus.
G.Eiw.i.U. zu Cr
GFR-CKD-EPI
GFR-IFCC
GFR-korr
GOT (AST) IFCC
GPT (ALT) IFCC
Gentamicin
Ges.-Eiw.i.U.
Gesamt-T3
Gesamt-T4
Gesamteiweiß
Glu
Glucose
Glucose i. Liq.
Glucose i. S./LH
Glucose im Urin
HCG u

...

CIC
CK IFCC
CK-MB 37C
CK-MB rel.
CMV-GS VR
COHb
CRP
Ca
Ca i. U. Aussch.
Ca++
Ca, korr. (Alb.)
Calcium i. U.
Chlorid i. U.
Chlorid i.U. Aus
Cholesterin
Cl
Cl-
Cortisol i. S.
Creatinin
Creatinin i.U.
Creatinin-Clear.
Cut Off HA
Cystatin C IFCC neu
D-Dimer
D-Dimer (Innov.)
D-Dimer (high)
D-Dimere (Innov) HA
D-Dimere vd 1:20 HA
Deseth.amiodaron
Digitoxin
Digoxin
EBV-DNA VR
ERY i. L.
EXTEM_Alpha Rotem
EXTEM_Alpha10 Rotem
EXTEM_CFT Rotem
EXTEM_CT Rotem
Eiw. Liquor
Epinephrin (5µM) HA
Ethanol
Extinktion HA
F V Leiden
F. VIII:C - high
F. XIII (CS5100)
FIBTEM_Ampl10 HA
FIO2
FSH 2
FT3
FVIII high HA
Faktor II HA
Faktor IX HA
Faktor V HA
Faktor VII HA
Faktor X HA
Faktor XI HA
Faktor XII HA
Faktor XIII HA
Fe
Ferritin 2
Ferritin-Index
Fib.n.C.(CS5100)
Fib/C HA
Fibrinogen
Fibrinogen Konz.
Freies T4
G.Eiw.i.U. Aus.
G.Eiw.i.U. zu Cr
GFR-CKD-EPI
GFR-IFCC
GFR-korr
GOT (AST) IFCC
GPT (ALT) IFCC
Gentamicin
Ges.-Eiw.i.U.
Gesamt-T3
Gesamt-T4
Gesamteiweiß
Glu
Glucose
Glucose i. Liq.
Glucose i. S./LH
Glucose i

-MCH
-MCHC
-MCV
-MTV
-Makrozytose
-Metamyeloz. % man
-Mikrozytose
-Monozyten %
-Monozyten % man
-Monozyten abs.
-Myelozyten % man
-Neutrophile %
-Neutrophile abs
-Path. Zyl. mik.
-Path. Zylinder
-Plasmazelle %
-Poikilozytose
-Polychromasie
-Promyeloz. % man
-Quotient Alb.
-Quotient IGG
-RBC i.U.
-RBC i.U. mikr. CH
-Rundepi mikr.
-Segmentkern. % man
-Stabkernige % man
-Thrombozyten
-Urobilinogen
-WBC i.U.
-pH i. U. CH
A-HBs VR
ABE
ACE-F
ACTH
ADH/Vasopressin
ADP (5 µM) HA
ADP (5) Revers. HA
ADP Citrat HA
AFP
AP IFCC
APC-Ratio
APCR-Ratio HA
ARQ
AT III
AT III (CS5100)
AaDpO2
Albumin
Aldosteron FV
Amiodaron
Ammoniak
Amylase IFCC
Amylase i. U. 25
Amylase i.U.37C
Anti Cardiolipin HA
Anti-FXa-Akt. HA
Anti-IIa Argatra HA
Anti-b2-Glycpr.I HA
Antithrombin HA
Arachidon Citrat HA
Arachidons.(0.5) HA
Bilirubin dir.
Bilirubin ges.
Bilirubin indir.
CHE 37C
CIC
CK IFCC
CK-MB 37C
CK-MB rel.
CMV-GS VR
COHb
CRP
Ca
Ca i. U. Aussch.
Ca++
Ca, korr. (Alb.)
Calcium i. U.
Chlorid i. U.
Chlorid i.U. Aus
Choleste

% unreife Thr.
- ADP [5 µM]
- ADP-Revers.
- Arach. [0.5mM]
- Epineph.[5 µM]
- Koll.[10µg/ml]
- PFA Coll./ADP
- PFA Coll./Epi.
- a. Ly. v. reak
-Albumin
-Albumin abs.
-Albumin in Liq.
-Albumin in S.
-Alpha-1-Glob.
-Alpha-1-Glob.ab
-Alpha-2-Glob.
-Alpha-2-Glob.ab
-Anisozytose
-Atyp. Lympho.% man
-Atyp. Zellen % man
-Bakterien
-Basophile %
-Basophile % man
-Basophile abs.
-Beta-Glob.
-Beta-Glob. abs.
-Blasten % man
-Blut (Ery.)
-Dichte
-EVB
-Eiweiß
-Eosinophile %
-Eosinophile % man
-Eosinophile abs
-Epithelien
-Erythrobl. % man
-Erythrozyten
-Gamma-Glob. 3182
-Gamma-Glob. abs 3183
-Hefe mikr.
-Hyal. Zyl.
-Hypochromasie
-Hämatokrit
-Hämoglobin
-IGA in Liquor
-IGA in Serum
-IGG Index
-IGG in Liquor
-IGG in Serum
-IGM in Liquor
-IGM in Serum
-Kernsch.in100%
-Leukos
-Leukozyten
-Lymphozyten %
-Lymphozyten % man
-Lymphozyten abs
-MCH
-MCHC
-MCV
-MTV
-Makrozytose
-Metamyeloz. % man
-Mikrozytose
-Monozyten %
-Monozyten % man
-Monozyten abs.
-Myelozyten % man
-Neutrophile %
-Neutrophile abs
-Path

here
% unreife Thr.
- ADP [5 µM]
- ADP-Revers.
- Arach. [0.5mM]
- Epineph.[5 µM]
- Koll.[10µg/ml]
- PFA Coll./ADP
- PFA Coll./Epi.
- a. Ly. v. reak
-Albumin
-Albumin abs.
-Albumin in Liq.
-Albumin in S.
-Alpha-1-Glob.
-Alpha-1-Glob.ab
-Alpha-2-Glob.
-Alpha-2-Glob.ab
-Anisozytose
-Atyp. Lympho.% man
-Atyp. Zellen % man
-Bakterien
-Basophile %
-Basophile % man
-Basophile abs.
-Beta-Glob.
-Beta-Glob. abs.
-Blasten % man
-Blut (Ery.)
-Dichte
-EVB
-Eiweiß
-Eosinophile %
-Eosinophile % man
-Eosinophile abs
-Epithelien
-Erythrobl. % man
-Erythrozyten
-Gamma-Glob. 3182
-Gamma-Glob. abs 3183
-Hefe mikr.
-Hyal. Zyl.
-Hypochromasie
-Hämatokrit
-Hämoglobin
-IGA in Liquor
-IGA in Serum
-IGG Index
-IGG in Liquor
-IGG in Serum
-IGM in Liquor
-IGM in Serum
-Kernsch.in100%
-Leukos
-Leukozyten
-Lymphozyten %
-Lymphozyten % man
-Lymphozyten abs
-MCH
-MCHC
-MCV
-MTV
-Makrozytose
-Metamyeloz. % man
-Mikrozytose
-Monozyten %
-Monozyten % man
-Monozyten abs.
-Myelozyten % man
-Neutrophile %
-Neutrophile abs


U-MikroALB
U-MikroALB 2
UFH-Hep. HA
Urin Sammelvol.
Urin Sammelzeit
VZV-GS VR
VZV-MS VR
Valproinsäure
Vancomycin
Vitamin B12
Zytoz.präp. Liq.
a/ApO2
aPTT (FS,CS5100)
aPTT FS HA
aPTT neu
aPTT570nm HA
alpha-HBDH 37C
dRVV Ratio norm.
extrazel.Hb HA
gamma-GT IFCC
korr. rel. Reti.
mOsm
p50(act)
pCO2
pCO2(T)
pH
pH(T)
pO2
pO2(T)
proBNP
sO2
sTrf-Rezeptor
tHb
tO2
vWF Akt.(CS5100)
vWF-Ag (CS5100)
vWF-Ag neu HA
vWF-Akt.n HA
vWF-Ratio
vWF-Ratio HA


In [98]:
print('first dimension should be:', len(pat_list), '(= number of feasible patients)')
print('second dimension should be:', days_interval * len(df[df.Kategorie=='Labor']['Wertbezeichner'].unique()), '(= days_interval * number of identifiers for category Labor)')
print('actual dimensuons are', X_labor_all.shape)

first dimension should be: 294 (= number of feasible patients)
second dimension should be: 1845 (= days_interval * number of identifiers for category Labor)
actual dimensuons are (294, 1845)


all patients, mean over hours not very meaningful

#### Stichprobentest

In [ ]:
index = list(set(map(operator.itemgetter(0), x_vital_all[x_vital_all.FallNr==test_pat].index.tolist())))
index = sorted(index)
print(index)
# note: some identifier might be missing (e.g. ICP for patient index)
for i in index:
    df_subset = x_vital_all[x_vital_all.FallNr==test_pat].loc[i]['Wert_norm'][:days_interval] #  = 5 
    print(df_subset)

In [ ]:
i = sorted(df.FallNr.unique()).index(test_pat)
X_vital_all[i]